In [21]:
from datasets import load_dataset
from datasets import Dataset
import pandas as pd
dataset = load_dataset('trec')
#display(dataset)
dataset = dataset.remove_columns("label-fine")
#display(dataset)
dataset = dataset.rename_column("label-coarse", "label")
display(dataset)


train_validation_ds = dataset['train'].shuffle(seed=42).select([i for i in list(range(5000))])
train_ds = Dataset.from_dict(train_validation_ds[0:4500])
validation_ds = Dataset.from_dict(train_validation_ds[4500:5000])
test_ds = dataset['test'].shuffle(seed=42).select([i for i in list(range(500))])




display(train_ds)
train_df = pd.DataFrame(train_ds)
display(train_df)

Using custom data configuration default
Reusing dataset trec (C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 668.41it/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 5452
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 500
    })
})

Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-bc761497303cb1ab.arrow
Loading cached shuffled indices for dataset at C:\Users\DELL\.cache\huggingface\datasets\trec\default\1.1.0\751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9\cache-d432b7eeb4042bcd.arrow


Dataset({
    features: ['label', 'text'],
    num_rows: 4500
})

,label,text
0,0,How can I transport files from one computer to...
1,3,Who are the two sons of Ozzie and Harriet Nels...
2,2,What does pH stand for ?
3,4,How many people does Honda employ in the U.S. ?
4,1,What newspaper returned a Pulitzer Prize for t...
...,...,...
4495,1,What does Final Four refer to in the sports wo...
4496,3,What is Nicholas Cage 's occupation ?
4497,1,"In order from the top , the four stripes on a ..."
4498,3,What kind of business is 7-Eleven ?


In [22]:
#wordnet part
import nltk
#nltk.download('omw-1.4')
from nltk.corpus import wordnet


from senticnet.senticnet import SenticNet
import pandas as pd

import spacy
from spacy import displacy
from lemminflect import getInflection

In [23]:
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.pre_transformation import RepeatModification
from textattack.constraints.pre_transformation import StopwordModification
from textattack.constraints.grammaticality.part_of_speech import PartOfSpeech
from textattack.constraints.semantics.bert_score import BERTScore

from textattack.augmentation import Augmenter
from textattack.augmentation.recipes import CheckListAugmenter





# Create augmenter with specified parameters
augmenter = CheckListAugmenter(pct_words_to_swap=0.1, transformations_per_example=1)

In [24]:
# Augment!
# additional parameters can be modified if not during initiation




def results_list_creation(s):
    try:
        results = augmenter.augment(s)
        
    
    except Exception as e:
        results = ['Error occured in text generation .']
    
    return results

import time
from multiprocessing import Pool
from concurrent import futures

time_sta = time.perf_counter()
results_list = []

#print(list(train_df['text']))

with futures.ThreadPoolExecutor() as executor:
    results_list_itera = executor.map(results_list_creation, list(train_df['text'])) 

for results in results_list_itera:
    results_list.append(results)



time_end = time.perf_counter()
tim = time_end - time_sta

In [25]:


print(tim/3600)




0.023928187694444443


In [26]:
display(results_list)

[['How can I transport files from two computer to another ?'],
 ['Who are the two sons of Ozzie and Harriet Gruber ?'],
 ['What does pH stand for ?'],
 ['How many people does Honda employ in the U.S. ?'],
 ["What newspaper returned a Pulitzer Prize for the fraudulent story Jimmy 's World ?"],
 ['What two cities usually mark the extremes of English Channel swims ?'],
 ["Who was the mother of the man who wouldn't be king , the duke of Windsor ?"],
 ['What chocolate company gives you a two kiss ?'],
 ['What is the temperature for cooking ?'],
 ['Name the Islamic counterpart to the Red Cross .'],
 ['What are my legal rights in an automobile repossession in California ?'],
 ['Where was helium first discovered , hence its name ?'],
 ["Whom does Uncle Stith 's girl friend , Honey , room with ?"],
 ['Who invented television ?'],
 ['What James Bostick book is subtitled Spanish Travels and Reflections ?'],
 ['What does laser stand for ?'],
 ['What is the Homelite Inc. home page ?'],
 ["What are 

In [27]:
import itertools

original_sentences_list = []

for i in range(len(train_df)):
    number_of_generated_sentences_per_original_sentence = len(results_list[i])
    original_sentences = [list(train_df['text'])[i] for j in range(number_of_generated_sentences_per_original_sentence)]
    original_sentences_list.append(original_sentences)
display(original_sentences_list)

original_sentences_list_1d = list(itertools.chain.from_iterable(original_sentences_list))
results_list_1d = list(itertools.chain.from_iterable(results_list))


[['How can I transport files from one computer to another ?'],
 ['Who are the two sons of Ozzie and Harriet Nelson ?'],
 ['What does pH stand for ?'],
 ['How many people does Honda employ in the U.S. ?'],
 ["What newspaper returned a Pulitzer Prize for the fraudulent story Jimmy 's World ?"],
 ['What two cities usually mark the extremes of English Channel swims ?'],
 ['Who was the mother of the man who would not be king , the duke of Windsor ?'],
 ['What chocolate company gives you a one-pound kiss ?'],
 ['What is the temperature for cooking ?'],
 ['Name the Islamic counterpart to the Red Cross .'],
 ['What are my legal rights in an automobile repossession in California ?'],
 ['Where was helium first discovered , hence its name ?'],
 ["Whom does Uncle Duke 's girl friend , Honey , room with ?"],
 ['Who invented television ?'],
 ['What James Michener book is subtitled Spanish Travels and Reflections ?'],
 ['What does laser stand for ?'],
 ['What is the Homelite Inc. home page ?'],
 ["Wh

In [28]:
import pandas as pd 
augmented_data = {'original_text':original_sentences_list_1d,'perturbed_text':results_list_1d}
augmented_data_df = pd.DataFrame(augmented_data)
display(augmented_data_df)


,original_text,perturbed_text
0,How can I transport files from one computer to...,How can I transport files from two computer to...
1,Who are the two sons of Ozzie and Harriet Nels...,Who are the two sons of Ozzie and Harriet Grub...
2,What does pH stand for ?,What does pH stand for ?
3,How many people does Honda employ in the U.S. ?,How many people does Honda employ in the U.S. ?
4,What newspaper returned a Pulitzer Prize for t...,What newspaper returned a Pulitzer Prize for t...
...,...,...
4495,What does Final Four refer to in the sports wo...,What does Final Four refer to in the sports wo...
4496,What is Nicholas Cage 's occupation ?,What is Deshun Cage 's occupation ?
4497,"In order from the top , the four stripes on a ...","In order from the top , the four stripes on a ..."
4498,What kind of business is 7-Eleven ?,What kind of business is twenty-two ?


In [29]:
#remove augmented_data_df['original_text'] == augmented_data_df['perturbed_text']
target = augmented_data_df.index[augmented_data_df['original_text'] == augmented_data_df['perturbed_text']]
augmented_data_df = augmented_data_df.drop(target)
augmented_data_df = augmented_data_df.reset_index(drop=True)
display(augmented_data_df)

,original_text,perturbed_text
0,How can I transport files from one computer to...,How can I transport files from two computer to...
1,Who are the two sons of Ozzie and Harriet Nels...,Who are the two sons of Ozzie and Harriet Grub...
2,Who was the mother of the man who would not be...,Who was the mother of the man who wouldn't be ...
3,What chocolate company gives you a one-pound k...,What chocolate company gives you a two kiss ?
4,"Whom does Uncle Duke 's girl friend , Honey , ...","Whom does Uncle Stith 's girl friend , Honey ,..."
...,...,...
1360,Name one of King Henry VIII 's wives .,Name one of King Henry Cornejo 's wives .
1361,What did Mighty Mouse always sing as he went i...,What did Mighty Marchant always sing as he wen...
1362,What is Nicholas Cage 's occupation ?,What is Deshun Cage 's occupation ?
1363,What kind of business is 7-Eleven ?,What kind of business is twenty-two ?


In [30]:
#save to csv file
augmented_data_df.to_csv( r'C:\Users\DELL\Text_Augmentation\trec_dataset_3runs\augumented_dataset\CheckList\run3\dataset.csv')